<a href="https://colab.research.google.com/github/anassaffo/Leupahan_University_Projects/blob/main/Course_Analysing_Networks_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mounting:

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Importing Packages

In [74]:
import sys
sys.path.append('/content/drive/MyDrive/AN')
import rafs_instance as instance
import json
import os
import os.path
from os import path
import xml.etree.ElementTree as ET
import networkx as nx
from lxml import etree
import pdb
from collections import defaultdict


# Loading data files:

In [48]:
layoutFile = r'/content/drive/MyDrive/AN/data/layout/1-1-1-2-1.xlayo'
# podInfoFile = '/content/drive/MyDrive/AN/data/sku24/pods_infos.txt'
podInfoFile = '/content/drive/MyDrive/AN/data/sku360/pods_infos.txt'

instances = {}
# instances[24,2] = r'/content/drive/MyDrive/AN/data/sku24/layout_sku_24_2.xml'
instances[360,2] = r'/content/drive/MyDrive/AN/data/sku360/layout_sku_360_2.xml'

storagePolicies = {}
storagePolicies['dedicated'] = '/content/drive/MyDrive/AN/data/sku24/pods_items_dedicated_1.txt'
# storagePolicies['mixed'] = '/content/drive/MyDrive/AN/data/sku24/pods_items_mixed_shevels_1-5.txt'

orders = {}
# orders['10_1x6']=r'/content/drive/MyDrive/AN/data/sku24/orders_10_mean_1x6_sku_24.xml'
# orders['10_5']=r'/content/drive/MyDrive/AN/data/sku24/orders_10_mean_5_sku_24.xml'
# orders['20_5']=r'/content/drive/MyDrive/AN/data/sku24/orders_20_mean_5_sku_24.xml'

orders['10_5']=r'/content/drive/MyDrive/AN/data/sku360/orders_10_mean_5_sku_360.xml'

## Exploring the XML files:

* first we read the XML file

In [ ]:
tree_etree = etree.parse(orders['10_5'])
tree_etree_instance = etree.parse(instances[24,2])

In [ ]:
tree = ET.parse(orders['10_5'])
tree

* now we see the parts of the file we have: XML has the biggest element as the root that contains all the children. inside each element there might be attributes that have rules for the element

In [ ]:
root = tree.getroot()
root_instance = tree_etree_instance.getroot()

In [ ]:
root.tag

In [ ]:
root_instance.tag

* we can simply explore our main xml elements inside the root with a simple for loop(attributes are inside the { }):

In [ ]:
for child in root:
    print(child.tag, child.attrib)

ItemDescriptions {}
ItemBundles {}
Orders {}


In [ ]:
for child in root_instance:
    print(child.tag, child.attrib)

Waypoints {}
Bots {}
ChargingStations {}
PickLocations {}
Pods {}
Elevators {}
InputStations {}
OutputStations {}
Tiers {}


* exploring every element in the tree:

In [ ]:
[elem.tag for elem in root.iter()]

* using the tostring() method to see the xml file:

In [ ]:
print(ET.tostring(root, encoding='utf8').decode('utf8'))

In [ ]:
print(ET.tostring(root_instance, encoding='utf8').decode('utf8'))

* lets see the attributes of each sub-element:

In [ ]:
print("Item Description:")
for item in root.iter('ItemDescription'):
    print(item.attrib)
print("*******","\n Position:")
for item in root.iter('Position'):
    print(item.attrib)
print("*******","\n Order:")
for item in root.iter('Order'):
    print(item.attrib)

In [ ]:
print("Way Point:\n")
for item in root_instance.iter('Waypoint'):
    print(item.attrib)
print("\n\n*******","\n Bot:\n")
for item in root_instance.iter('Bot'):
    print(item.attrib)
print("\n\n*******","\n Pick Location:\n")
for item in root_instance.iter('PickLocation'):
    print(item.attrib)
print("\n\n*******","\n Output Station:\n")
for item in root_instance.iter('OutputStation'):
    print(item.attrib)

* now we look into the content of elements:

In [ ]:
positions = tree_etree.xpath("//Position[@Count='1']")

for position in positions:
  print(etree.tostring(position))

In [ ]:
positions = tree_etree.xpath("//ItemDescription[@Letter='a']")

for position in positions:
  print(etree.tostring(position))

In [ ]:
lst = tree_etree_instance.xpath("//Waypoint[@ChargingStation='-1']")
print('User count:', len(lst)) # this is how long is it

for item in lst:
  print(etree.tostring(item))
#for item in lst:
 #   print('Path', item.find('Paths').text)

## JSON Exploration:

* let's first load the file

In [ ]:
json_names = ["layout_sku_24_2", "layout_sku_360_2", "layout", "1-1-1-5-240 - Modified 16 - 2 depot", "1-1-1-5-240Mod16"]
file_path = r'/content/drive/MyDrive/AN/data/distances/' + json_names[0] + '.json'

In [ ]:
with open(file_path, "r") as read_file:
    data = json.load(read_file)

In [ ]:
data

{'0': {'0': 0,
  '1': 0.9000000000000004,
  '10': 3.5999999999999996,
  '11': 4.5,
  '12': 12.5,
  '13': 13.4,
  '14': 14.3,
  '15': 15.200000000000001,
  '16': 16.1,
  '17': 17.0,
  '18': 12.5,
  '19': 13.4,
  '2': 1.8000000000000007,
  '20': 14.3,
  '21': 15.200000000000001,
  '22': 16.1,
  '23': 17.0,
  '3': 2.700000000000001,
  '4': 3.5999999999999996,
  '5': 4.5,
  '6': 0,
  '7': 0.9000000000000004,
  '8': 1.8000000000000007,
  '9': 2.700000000000001,
  'OutD0': 9.049999999999999,
  'OutD1': 13.55},
 '1': {'0': 0.9000000000000004,
  '1': 0,
  '10': 2.6999999999999993,
  '11': 3.5999999999999996,
  '12': 13.399999999999999,
  '13': 14.299999999999999,
  '14': 15.2,
  '15': 16.1,
  '16': 17.0,
  '17': 16.1,
  '18': 13.399999999999999,
  '19': 14.299999999999999,
  '2': 0.9000000000000004,
  '20': 15.2,
  '21': 16.1,
  '22': 17.0,
  '23': 16.1,
  '3': 1.8000000000000007,
  '4': 2.6999999999999993,
  '5': 3.5999999999999996,
  '6': 0.9000000000000004,
  '7': 0,
  '8': 0.90000000000000

In [ ]:
print(data["OutD0"]["0"])
print(data["OutD1"]["0"])

9.049999999999999
13.55


# Data Processing:

In [49]:
class WarehouseDateProcessing():
    def __init__(self, warehouseInstance, batch_size = None):
        self.Warehouse = warehouseInstance # the link to the .py file
        self._InitSets(warehouseInstance, batch_size) # creates V as a dict with output station and item lists based on color and letter for each order   
        
    def preprocessingFilterPods(self, warehouseInstance):
        
        resize_pods = {}
        print("preprocessingFilterPods")
        item_id_list=[]
        for order in warehouseInstance.Orders:
            for pos in order.Positions.values():
                item = warehouseInstance.ItemDescriptions[pos.ItemDescID].Color.lower() + '/' + warehouseInstance.ItemDescriptions[pos.ItemDescID].Letter
                #weight = warehouseInstance.ItemDescriptions[pos.ItemDescID].weight
                #print(item)
                #item_id = pos.ItemDescID
                if item not in item_id_list:
                    item_id_list.append(item)
                    #print(item_id)
       
        #for item in item_id_list:
        #    print(item)

        # for dedicated
        for pod in warehouseInstance.Pods.values():
            for item in pod.Items:
                #print("item in pod.Items:", item.ID) # each item.ID is the letter and color of that item
                if item.ID in item_id_list:
                    #print("item.ID in item_id_list:", item.ID)
                    resize_pods[pod.ID] = pod
        
        #print(resize_pods)
        return resize_pods

    # Initialize sets and parameters           
    def _InitSets(self,warehouseInstance, batch_size):
        #V Set of nodes, including shelves V^S and stations (depots)
        # V^D (V=V^S U V^D)
        #Add output and input depots
        
        self.V__D__C = warehouseInstance.OutputStations
        ##self.V__D__F = warehouseInstance.InputStations
        self.V__D__F = {}
        #depot = ('D999', )
        #Old self.V__D = {'D999':depot}
             
        self.V__D = {**self.V__D__C, **self.V__D__F}
        
        #hli
        #self.V__S = warehouseInstance.Pods
        self.V__S = self.preprocessingFilterPods(warehouseInstance)
        
        #Merge dictionaries
        self.V = {**self.V__D, **self.V__S}

    def CalculateDistance(self):

        file_path = r'/content/drive/MyDrive/AN/data/distances/' + os.path.splitext(os.path.basename(self.Warehouse.InstanceFile))[0] + '.json'
        if not path.exists(file_path):
            #Create d_ij
            #d_ij = tupledict()
            d_ij = {}
            #print(self.V.items)
            for key_i, node_i in self.V.items():
                for key_j, node_j in self.V.items():  
                    
                    source = 'w'+node_i.GetPickWaypoint().ID
                    target = 'w'+node_j.GetPickWaypoint().ID
                    
                    #Calc distance with weighted shortest path
                    d_ij[(key_i,key_j)] = nx.shortest_path_length(self.Graph, source=source, target=target, weight='weight')
            
            #Parse and save
            d_ij_dict = {}
            for key,value in d_ij.items():
                i,j = key  
                if i not in d_ij_dict:
                    d_ij_dict[i]={} 
                d_ij_dict[i][j] = value
            
            with open(file_path, 'w') as fp:
                json.dump(d_ij_dict, fp)
        
        else: 
            #Load and deparse
            with open(file_path, 'r') as fp:
                d_ij_dict = json.load(fp)
            print('d_ij file %s loaded'%(file_path)) 
                
            #d_ij = tupledict()
            d_ij = {}
            for i, values in d_ij_dict.items():
                for j, dist in values.items():
                    d_ij[i,j] = dist                
                
        return d_ij
		

class Demo():
    def __init__(self, splitOrders = False):
        
        self.batch_weight = 18
        self.item_id_pod_id_dict = {}
        #[0]
        self.warehouseInstance = self.prepareData()
        self.distance_ij = self.initData()
        #[2]
        if storagePolicies.get('dedicated'):
            self.is_storage_dedicated = True
        else:
            self.is_storage_dedicated = False


	# warehouse instance
    def prepareData(self):
        print("[0] preparing all data with the standard format: ")
        #print(instances)
        #Every instance
        for key,instanceFile in instances.items():
          # the key is the previously defined tuple in instances
          # the instanceFile is the XML path
            #print(instanceFile)
            podAmount = key[0]
            depotAmount = key[1] 
            #For different orders
            for key, orderFile in orders.items():
              # the key is the previously defined tuple in orders
                orderAmount = key
                #For storage policies
                for storagePolicy, storagePolicyFile in storagePolicies.items(): # what item in what pod   
                    warehouseInstance = instance.Warehouse(layoutFile, instanceFile, podInfoFile, storagePolicyFile, orderFile)
        return warehouseInstance

	# distance
    def initData(self):
        print("[1] changing data format for the algorithm we used here: ")
        warehouse_data_processing = WarehouseDateProcessing(self.warehouseInstance)
        #Distance d_ij between two nodes i,j \in V
        d_ij = warehouse_data_processing.CalculateDistance()
        return d_ij


if __name__ == "__main__":
    _demo = Demo()	
    print("todo:")


[0] preparing all data with the standard format: 
[1] changing data format for the algorithm we used here: 
preprocessingFilterPods
d_ij file /content/drive/MyDrive/AN/data/distances/layout_sku_360_2.json loaded
todo:


# Greedy


## Distances:

In [ ]:
### get nested distances dict from json file
with open('/content/drive/MyDrive/AN/data/distances/layout_sku_24_2.json') as json_file:
    distances = json.load(json_file)

packing_stations = _demo.warehouseInstance.OutputStations.keys() # getting the packing stations as dict


## Functions:

In [62]:
def find_nearset_packing_station(order):
  order_distance_sum = {}
  order_positions = extract_order_positions(order)
  for packing_station in packing_stations:
    order_distance_sum[packing_station] = 0 # the distance from the packing station at the start is 0
    # for pos, _ in order.Positions.items():
    #   order_distance_sum[packing_station] += distances[pos][packing_station]
    for pos in order_positions:
      order_distance_sum[packing_station] += distances[pos][packing_station] # calculate the sum of distances from each position to the other in the order

  nearest_packing, sum_distance = sorted(order_distance_sum.items(), key=lambda x: x[1])[0] # only select the shortest distance
  return nearest_packing, sum_distance


def greedy_order_to_packing_station_assingment(orders):
  assinged_packing_stations = {}
  for order in orders:
    nearest_packing, _ = find_nearset_packing_station(order) 
    assinged_packing_stations[order] = nearest_packing # have a dic of each order with its assigned packing station(greedy way)

  return assinged_packing_stations


def find_best_route(order_items_positions, packing_station_position):
  start = packing_station_position # start at the packing station
  head = start # the head of the path is the start
  path = [head] # keep track of the locations visited
  remaining = order_items_positions # a dict. of all the locations not yet visited in the order
  distance = 0.0
  time = 0.0
  # count = 0
  # prep_time = 30
  # unload_time = 20
  while len(remaining) > 0: # loop until all locations are visited in the order
    ds = {}
    for position in remaining: 
      # pdb.set_trace()
      ds[position] = distances[head][position]
      if position != packing_station_position:
        time += 3.0
        # count += 1
    best, best_distance = sorted(ds.items(), key=lambda x: x[1])[0] # select shortest path
    remaining.remove(best) # remove the location from the not visited dict.
    head = best # update the head of the path to the current location of the cobot
    path += [best] # update the path taken by the cobot
    distance += best_distance # update the distance traveled by the cobot
  distance += distances[best][start] # add the distance for going back to the packing station
  path += [start] # adding the packing station to the path
  # time += prep_time + unload_time
  return path, distance, time


# to extract the dict of all locations in an order
def extract_order_positions(order):
  return list(order.Positions.keys())


# to extract all the weights of the items inside an order
def extract_order_weight(order,ItemDescriptions):
  total_weight = 0
  for pos in order.Positions.values():
    total_weight += ItemDescriptions[pos.ItemDescID].Weight * float(pos.Count)
  return total_weight

## Assignments:

## Orders to packing stations

In [67]:
# assign the order to the packing station:
order_packing_assingments = greedy_order_to_packing_station_assingment(_demo.warehouseInstance.Orders)

packing_station_orders = defaultdict(list)
for order, packing_station in order_packing_assingments.items():
  packing_station_orders[packing_station] += [order]

## Orders to cobots:

In [73]:
station_0_orders = packing_station_orders['OutD0'].copy() # select a packing station

station_orders = station_0_orders
max_weight = 18

while len(station_orders) > 0: # loop until all orders are done
  greedy_batch = []
  batch_items = []

  total_weight = 0.0
  total_distance = 0.0
  total_time = 0.0

  while total_weight < max_weight and len(station_orders) > 0 : # keeping the constraints
    order_distances = {}
    order_times = {}
    time = 0.0
    for order in station_orders: # for every order in the asssigned orders to the station
      order_positions = extract_order_positions(order) # dict. of all the locations in the order
      _ , distance, time = find_best_route(order_positions + batch_items, 'OutD0') # select best route and make sure not to go back
      order_distances[order] = distance
      order_times[order] = time

      # _ , best_order_time = sorted(order_times.items(), key=lambda x: x[1])[0]

    best_order_found = False
    for best_order, best_order_distance in sorted(order_distances.items(), key=lambda x: x[1]): # loop through the best orders based on distance
      # best_order_time += (distance/2.0)
      best_order_weight = extract_order_weight(best_order,_demo.warehouseInstance.ItemDescriptions) # total weight in the order

      # print (total_weight, best_order_weight)
      # pdb.set_trace()
      if total_weight + best_order_weight <= max_weight: # keeping the weight constraint
        total_weight += best_order_weight
        total_distance += best_order_distance 
        greedy_batch += [best_order] # list of the best orders in the greedy way
        batch_items += order_positions
        #total_time += best_order_time
        best_order_found = True # to make sure we go through every order
        break

    # this will be true when the weight constraint is broken
    if not best_order_found: 
      break

    station_orders.remove(best_order) # remove the order that is finished
  print ("Order:\n", greedy_batch,"\nWeight:",  total_weight,"\n")


Order:
 [<rafs_instance.Order object at 0x7fbf6cb1bc50>, <rafs_instance.Order object at 0x7fbf6cb1bc90>, <rafs_instance.Order object at 0x7fbf6cb1b310>] 
Weight: 16.45 

Order:
 [<rafs_instance.Order object at 0x7fbf6cb1d790>, <rafs_instance.Order object at 0x7fbf6cb1e2d0>] 
Weight: 16.119999999999997 

Order:
Weight: 14.709999999999999 

Order:
Weight: 14.969999999999999 



## next steps:
* adding time
* *select orders in the list of orders for batch assignment (Solved)*
* balance the order assignment to the packing station
* use mutiple cobots




## Assigning the orders to the depots (with simulated data)

In [ ]:
#Assigning orders to depots
import numpy as np
import math

#We need following data:
# Dictionary with distances from depots to shelves: dict[depot][shelve] = distance
# Dictionary where items of the orders assigned to the shelves: dict[order][item] = shelve
# Dictionary with weights of items: dict[order][item] = weight - to calculate weight of orders for balancing
# I will need to create two dictionaries with d1 and d2: dict[order][item] = distance, for all depots
# Then calculate Mu = d1 - d2
#sort orders according to Mu
# calculate B - minimum number of batches
# Assign orders to depots

V_D = ['Depot' + str(i) for i in range(1,3)]
V_S = ['shelve' + str(i) for i in range(1,25)]
dist_shelves = (np.random.uniform(low=3, high=20, size=24).round(decimals = 2)).tolist()
#dictionary with distances assigned to each shelve:
dict_dist_shelves= dict(zip(V_D, dist_shelves))

orders = ['order_' + str(i) for i in range(1,4)]
items = ['item' + str(i) for i in range(1,8)]
dist = (np.random.uniform(low=5, high=20, size=24).round(decimals = 2)).tolist()
weights1 = (np.random.uniform(low=0.8, high=11, size=7).round(decimals = 2)).tolist()
weights2 = (np.random.uniform(low=0.2, high=9, size=7).round(decimals = 2)).tolist()
weights3 = (np.random.uniform(low=0.1, high=16, size=7).round(decimals = 2)).tolist()

#dictionary with weights assigned to each item:
dict_weights = {'order_1' :dict(zip(items, weights1)), 'order_2' :dict(zip(items, weights2)), 'order_3' :dict(zip(items, weights3))}


##dist_items = np.sample(dist_shelves, 7)
#depo1 items - distances:
dict_item_depo_1 = {'order_1' :dict(zip(items, dist_shelves[0:8])), 'order_2' :dict(zip(items, dist_shelves[4:12])), 'order_3' :dict(zip(items, dist_shelves[8:16]))}
dict_item_depo_2 = {'order_1' :dict(zip(items, dist_shelves[2:10])), 'order_2' :dict(zip(items, dist_shelves[14:22])), 'order_3' :dict(zip(items, dist_shelves[9:17]))}
D_o = zip
'''
output = {
    order: dict(zip(item, value))
    for order, item, value in zip(
        orders,
        items,
        [values_a, values_b, values_c, values_d]
)}
'''
#Initialization
#ωo_i = o:
V_D_number = [1, 2]
ωo_i_init = [0 for i in range(len(orders))]
ωo_i = {
    depot: dict(zip(orders, ωo_i_init))
    for depot in V_D_number} #assigning all orders to non existing depot (0)
weight_orders = dict(zip(orders,[sum(dict_weights[order].values()) for order in orders]))
weight_stations =  [0 for i in V_D]
weight_stations_dict = dict(zip(V_D_number,weight_stations))

#let max capacity be:
c = 18
#Min batches for all orders:
B_star = math.ceil(sum(orders_weights)/c)
#creating list of mu:
mu_list = []
priority_list = []
priority = 0
mu = 0
for order in dict_item_depo_1.keys():
    d1 = sum(dict_item_depo_1[order].values())
    d2 = sum(dict_item_depo_2[order].values())
    if d1 > d2:
        mu = d1 - d2
        priority = 2
    else:
        mu = d2 - d1
        priority = 1
    mu_list.append(mu)
    priority_list.append(priority)
lists_to_sort = zip(mu_list, orders, priority_list)
sorted_lists = sorted(lists_to_sort, reverse=True )#sorted according to first list
orders_sorted = [elem for _1, elem, _3 in sorted_lists]
mu_list_sorted = [elem for elem, _2, _3 in sorted_lists]
priority_list_sorted = [elem for _1, _2, elem in sorted_lists]
#assigning of beta:
for index, order in enumerate(orders_sorted):
    depot_number = priority_list_sorted[index]
    if math.ceil((weight_stations_dict[depot_number]+weight_orders[order])/c) <= math.ceil(B_star/len(V_D)):
        ωo_i[depot_number][order] = 1 #assigning the order to depot
        weight_stations_dict[depot_number] = weight_stations_dict[depot_number] + weight_orders[order]
        #orders_sorted.remove(order)
    #Version below is suitable only for two depots setting (temporary), I will create another one for layout360
    else:
        ind = V_D_number.index(depot_number)
        depot_number = V_D_number[ind - 1]
        ωo_i[depot_number][order] = 1  # assigning the order to depot
        weight_stations_dict[depot_number] = weight_stations_dict[depot_number] + weight_orders[order]

NameError: ignored

# Model Definition
- Sets, Parameters and Variables
- a shelf = demand node
- packing station = depot/supply node
- batch = vehicle
- 

## Tasks
- Greedy heuristic
- Simmulated Annealing (design a neighborhood to generate a solution s' given a solution s)
- Perturbation strategy for an interated local search

In [ ]:
### get nested distances dict from json file
with open('/content/drive/MyDrive/AN/data/distances/layout_sku_24_2.json') as json_file:
    distances = json.load(json_file)
    
    print("Type", type(distances))

Type <class 'dict'>


In [ ]:
# distance dictionary for e.g. Depot 1 to all other shelves and depot 2
distances["OutD0"]

{'0': 9.049999999999999,
 '1': 9.95,
 '10': 12.649999999999999,
 '11': 13.549999999999999,
 '12': 10.35,
 '13': 11.25,
 '14': 12.15,
 '15': 13.05,
 '16': 13.95,
 '17': 14.85,
 '18': 10.35,
 '19': 11.25,
 '2': 10.85,
 '20': 12.15,
 '21': 13.05,
 '22': 13.95,
 '23': 14.85,
 '3': 11.75,
 '4': 12.649999999999999,
 '5': 13.549999999999999,
 '6': 9.049999999999999,
 '7': 9.95,
 '8': 10.85,
 '9': 11.75,
 'OutD0': 0,
 'OutD1': 22.599999999999998}

In [ ]:
# get distance from depot 1 to shelf 1
distances["OutD1"]["OutD0"]

22.599999999999998

In [ ]:
distances["OutD0"]

{'0': 9.049999999999999,
 '1': 9.95,
 '10': 12.649999999999999,
 '11': 13.549999999999999,
 '12': 10.35,
 '13': 11.25,
 '14': 12.15,
 '15': 13.05,
 '16': 13.95,
 '17': 14.85,
 '18': 10.35,
 '19': 11.25,
 '2': 10.85,
 '20': 12.15,
 '21': 13.05,
 '22': 13.95,
 '23': 14.85,
 '3': 11.75,
 '4': 12.649999999999999,
 '5': 13.549999999999999,
 '6': 9.049999999999999,
 '7': 9.95,
 '8': 10.85,
 '9': 11.75,
 'OutD0': 0,
 'OutD1': 22.599999999999998}

In [ ]:
############ xml.files

In [ ]:
# create dictionary with dict[item][weight]
# in the same way you can access any item information from the order file like:
# dict[item][color] or dict[item][letter]

# read xml. file
#tree
tree_order_10 = ET.parse(orders['10_5'])
root_order_10 = tree_order_10.getroot()

TypeError: ignored

In [ ]:
# single root elements containing all other elements
root_order_10.tag

NameError: ignored

In [ ]:
# root has no attributes
root_order_10.attrib

{}

In [ ]:
# children of root "OrderList"
for child in root_order_10:
  print(child.tag, child.attrib)

ItemDescriptions {}
ItemBundles {}
Orders {}


In [ ]:
## this gives us all the items of the orders
#  but we need to know which items belongs to which order when iterating over the orders
for item in root_order_10.iter("Position"):
  print(item.attrib)

{'Count': '1', 'ItemDescriptionID': '3'}
{'Count': '1', 'ItemDescriptionID': '7'}
{'Count': '1', 'ItemDescriptionID': '7'}
{'Count': '2', 'ItemDescriptionID': '6'}
{'Count': '1', 'ItemDescriptionID': '8'}
{'Count': '2', 'ItemDescriptionID': '0'}
{'Count': '1', 'ItemDescriptionID': '2'}
{'Count': '1', 'ItemDescriptionID': '7'}
{'Count': '1', 'ItemDescriptionID': '5'}
{'Count': '2', 'ItemDescriptionID': '7'}
{'Count': '1', 'ItemDescriptionID': '1'}
{'Count': '1', 'ItemDescriptionID': '5'}
{'Count': '1', 'ItemDescriptionID': '6'}
{'Count': '2', 'ItemDescriptionID': '3'}
{'Count': '1', 'ItemDescriptionID': '5'}
{'Count': '1', 'ItemDescriptionID': '4'}
{'Count': '1', 'ItemDescriptionID': '5'}
{'Count': '3', 'ItemDescriptionID': '2'}
{'Count': '2', 'ItemDescriptionID': '4'}
{'Count': '1', 'ItemDescriptionID': '7'}
{'Count': '1', 'ItemDescriptionID': '8'}
{'Count': '3', 'ItemDescriptionID': '0'}
{'Count': '1', 'ItemDescriptionID': '3'}
{'Count': '1', 'ItemDescriptionID': '1'}
{'Count': '1', '

In [ ]:
############ here starts the important part

In [ ]:
# iterate over attributes and store them in a dictionary
item_description = {}
i = 0
for item in root_order_10.iter('ItemDescription'):
    item_description.update({f'item{i}': item.attrib})
    i += 1

In [ ]:
item_description

{'item0': {'Color': 'blue',
  'ID': '0',
  'Letter': 'a',
  'Type': 'Letter',
  'Weight': '2.94'},
 'item1': {'Color': 'green',
  'ID': '1',
  'Letter': 'a',
  'Type': 'Letter',
  'Weight': '2.12'},
 'item10': {'Color': 'green',
  'ID': '10',
  'Letter': 'd',
  'Type': 'Letter',
  'Weight': '0.58'},
 'item11': {'Color': 'red',
  'ID': '11',
  'Letter': 'd',
  'Type': 'Letter',
  'Weight': '2.79'},
 'item12': {'Color': 'blue',
  'ID': '12',
  'Letter': 'e',
  'Type': 'Letter',
  'Weight': '1.08'},
 'item13': {'Color': 'green',
  'ID': '13',
  'Letter': 'e',
  'Type': 'Letter',
  'Weight': '2.4'},
 'item14': {'Color': 'red',
  'ID': '14',
  'Letter': 'e',
  'Type': 'Letter',
  'Weight': '1.89'},
 'item15': {'Color': 'blue',
  'ID': '15',
  'Letter': 'f',
  'Type': 'Letter',
  'Weight': '2.64'},
 'item16': {'Color': 'green',
  'ID': '16',
  'Letter': 'f',
  'Type': 'Letter',
  'Weight': '2.21'},
 'item17': {'Color': 'red',
  'ID': '17',
  'Letter': 'f',
  'Type': 'Letter',
  'Weight': '0.

In [ ]:
# get information for each item
# weight for item with ID=0 is 2.94
# WE HAVE TO CHECK HERE; WHY THE WEIGHTS ARE NOT THE SAME AS IN THE FILE BUT ALL OTHERS VALUES (COLOR; LETTER) ARE THE SAME
item_description["item0"]["Weight"]

'2.94'

In [ ]:
# in the same way we can extract information about the order
# needs to be adjusted TBD
orders = {}
i = 0
for item in root_order_10.iter('Positions'):
  orders.update({f'item{i}': item.attrib})
  i += 1

In [ ]:
orders

{'item0': {},
 'item1': {},
 'item2': {},
 'item3': {},
 'item4': {},
 'item5': {},
 'item6': {},
 'item7': {},
 'item8': {},
 'item9': {}}

In [ ]:
# in the same way we can extract information about the order
# needs to be adjusted TBD
orders = {}
inner_dict = {}
outer_dict = {}
i = 0
j = 0
for Position in root_order_10.iter('Positions'):
  outer_dict.update({f'order{i}': {"item":"information"}})
  for item in Position:
    inner_dict.update({f'item{j}': item.attrib})
    outer_dict[i]=inner_dict
    j += 1
  i += 1

In [ ]:
outer_dict[1]